In [1]:
import ads
import os

In [8]:
# read the environment variable ADS_DEV_KEY
try:
    ads.config.token = os.environ['ADS_DEV_KEY']
    print('ADS_DEV_KEY is set')
except KeyError:
    print('Please set the ADS_DEV_KEY environment variable')
    print('For more info, see https://ads.readthedocs.io/en/latest/')

ADS_DEV_KEY is set


In [4]:
papers = ads.SearchQuery(q="supernova", sort="citation_count")

In [ ]:
for paper in papers:
    print(paper.title)